#### Import Modules

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

#### Import Data

In [2]:
file1 = pd.read_csv('../data/raw/File1.txt', header=None, sep=' ', names=['MeterID', 'codeDateTime', 'kWh'])


In [3]:
file_1 = file1.copy()

In [4]:
file_1.head()

,MeterID,codeDateTime,kWh
0,1392,19503,0.140
1,1392,19504,0.138
2,1392,19505,0.140
3,1392,19506,0.145
4,1392,19507,0.145


In [5]:
file_1['MeterID'].nunique()

1000

In [10]:
def code_to_datetime(code):
    if len(str(code)) != 5:
        raise ValueError("Input code must be a 5-digit integer.")

    day_code = int(str(code)[:3])
    time_code = int(str(code)[3:5])

    # Calculate the date
    base_date = datetime(2009, 1, 1)
    delta = timedelta(days=day_code)
    target_date = base_date + delta

    # Calculate the time
    hours = (time_code) // 2
    minutes = 30 * (time_code % 2)

    target_time = timedelta(hours=hours, minutes=minutes)

    # Combine the date and time to create the datetime object
    result_datetime = target_date + target_time

    return result_datetime

In [13]:
# Use tqdm to track progress
tqdm.pandas(desc="Converting")

# Apply the conversion function to the 'code' column
file_1['DateTime'] = file_1['codeDateTime'].progress_apply(code_to_datetime)

Converting: 100%|██████████| 24465838/24465838 [00:42<00:00, 575236.07it/s]


In [14]:
file_1.head(20)

,MeterID,codeDateTime,kWh,DateTime
0,1392,19503,0.140,2009-07-15 01:30:00
1,1392,19504,0.138,2009-07-15 02:00:00
2,1392,19505,0.140,2009-07-15 02:30:00
3,1392,19506,0.145,2009-07-15 03:00:00
4,1392,19507,0.145,2009-07-15 03:30:00
5,1392,19501,0.157,2009-07-15 00:30:00
6,1392,19502,0.144,2009-07-15 01:00:00
7,1392,19724,0.128,2009-07-17 12:00:00
8,1392,19725,0.142,2009-07-17 12:30:00
9,1392,19726,0.145,2009-07-17 13:00:00


In [43]:
def group_by_meter_id(df):
    # Initialize an empty DataFrame with 'DateTime' as the index
    unique_datetimes = df['DateTime'].unique()
    new_df = pd.DataFrame(index=unique_datetimes)

    # Iterate through MeterIDs and populate the new DataFrame with progress tracking
    meter_ids = df['MeterID'].unique()
    for meter_id in tqdm(meter_ids, desc="Processing MeterIDs"):
        meter_data = df[df['MeterID'] == meter_id]
        new_df[meter_id] = new_df.index.map(
            lambda dt: meter_data[meter_data['DateTime'] == dt]['kWh'].values[0] if len(meter_data[meter_data['DateTime'] == dt]['kWh']) > 0 else np.nan
        )

    # Reset the index of the new DataFrame
    new_df.reset_index(inplace=True)
    new_df.set_index('index', inplace=True)

    return new_df

In [47]:
grouped_data = group_by_meter_id(file_1[:3000])

Processing MeterIDs: 100%|██████████| 11/11 [00:00<00:00, 13.16it/s]


In [48]:
grouped_data.head(20).sort_index()

,1392,1951,1491,1194,1804,1048,1802,1287,1529,1463,1860
index,,,,,,,,,,,
2009-07-15 00:30:00,0.157,0.165,0.019,0.399,0.096,0.148,0.910,0.840,0.407,0.138,NaN
2009-07-15 01:00:00,0.144,0.112,0.112,0.305,0.126,0.225,0.514,0.986,0.421,0.337,NaN
2009-07-15 01:30:00,0.140,0.014,0.018,0.280,0.086,0.200,0.351,1.432,0.281,0.223,NaN
2009-07-15 02:00:00,0.138,0.014,0.011,0.245,0.123,0.143,0.243,1.576,0.142,0.280,NaN
2009-07-15 02:30:00,0.140,0.041,0.111,0.389,0.100,0.200,0.258,1.428,0.208,0.159,NaN
2009-07-15 03:00:00,0.145,0.115,0.018,0.102,0.181,0.248,0.284,1.268,0.233,0.163,NaN
2009-07-15 03:30:00,0.145,0.069,0.013,0.139,0.178,0.160,0.330,0.966,0.079,0.195,NaN
2009-07-17 12:00:00,0.128,0.112,0.329,2.800,0.116,1.867,0.152,1.335,0.204,0.186,NaN
2009-07-17 12:30:00,0.142,0.054,1.385,1.358,0.332,0.530,0.141,0.949,0.202,0.314,NaN


In [49]:
grouped_data.to_csv('../data/interim/grouped_data.csv', index=True, header=True)